In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import time, datetime
from utils import *

from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# modified 0211

In [4]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/Duolingo/learning_traces.csv'
log = pd.read_csv(bath_path, encoding = "utf-8",low_memory=False)

In [5]:
log.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1


In [6]:
log.columns

Index(['p_recall', 'timestamp', 'delta', 'user_id', 'learning_language',
       'ui_language', 'lexeme_id', 'lexeme_string', 'history_seen',
       'history_correct', 'session_seen', 'session_correct'],
      dtype='object')

In [7]:
check_nan(log)
count_unique(log, ['user_id', 'learning_language', 'lexeme_id'])
print('total number of logs: ', len(log))

number of NaN in column p_recall: 0
number of NaN in column timestamp: 0
number of NaN in column delta: 0
number of NaN in column user_id: 0
number of NaN in column learning_language: 0
number of NaN in column ui_language: 0
number of NaN in column lexeme_id: 0
number of NaN in column lexeme_string: 0
number of NaN in column history_seen: 0
number of NaN in column history_correct: 0
number of NaN in column session_seen: 0
number of NaN in column session_correct: 0
number of unique value in user_id: 115222
number of unique value in learning_language: 6
number of unique value in lexeme_id: 19279
total number of logs:  12854226


In [11]:
user1 = log.loc[log.user_id == 'u:FO']
len(user1.lexeme_id.unique())

12

In [17]:
remove_thres = 200

log = log[~log.lexeme_id.isna()]
log = remove_log(log, 'user_id', num=remove_thres)
print(len(log))

8449234


In [18]:
# create new data
# choose threshood 200
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = log.user_id
df.problem_id = log.lexeme_id
df.skill_id = log.lexeme_id
df.timestamp = log.timestamp
df.dwell_time = log.delta
df.correct = log.p_recall * 1

In [19]:
df = df.sort_values('timestamp', ascending=True)
df.head()

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
87,u:dwbJ,1052c3ace653dbc8923eaa183bc02b88,1052c3ace653dbc8923eaa183bc02b88,1362082504,357,1.0
88,u:dwbJ,9cba1b30f88bf3c047b22cffcaf88c12,9cba1b30f88bf3c047b22cffcaf88c12,1362082504,357,1.0
89,u:dwbJ,961cd149f20f2571419b1412d849f19a,961cd149f20f2571419b1412d849f19a,1362082504,357,1.0
90,u:dwbJ,5cbb1249562e95794a4c4ae0e2d8ae26,5cbb1249562e95794a4c4ae0e2d8ae26,1362082504,357,0.8
91,u:dwbJ,2df65bdf80d10d2b78d62cb2e0a731d8,2df65bdf80d10d2b78d62cb2e0a731d8,1362082504,357,1.0


In [20]:
# re-index
# - skill
skill_id = list(df['skill_id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
df['skill_id'] = df['skill_id'].apply(lambda x: skill_dict[x])

# - problem
pro_id = list(df['problem_id'].unique())
pro_dict = dict(zip(pro_id, range(len(pro_id))))
df['problem_id'] = df['problem_id'].apply(lambda x: pro_dict[x])

# - student
stu_id = list(df['user_id'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
df['user_id'] = df['user_id'].apply(lambda x: stu_dict[x])

In [30]:
df = df.sort_values('timestamp', ascending=True)
df.head()
df = df.astype({
    'timestamp': np.float64,
    'dwell_time': np.float64,
    'correct': np.float64,
    'problem_id': np.int64,
    'skill_id': np.int64,
    'user_id': np.int64
    })

In [31]:
# Save
df.to_csv('/mnt/qb/work/mlcolab/hzhou52/kt/Duolingo/'+'interactions_{}.csv'.format(remove_thres), sep='\t', index=False)